# 04 - Model Building

In [83]:
train_dataset_size = 0.7 #% of the data that is used for training to speed up the process of finding the best model
should_build_model = False #If False, the model will be loaded from a file.
should_save_model = True #If True, the model will be saved to a file.

## Setup

In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import yaml
import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
# pd.set_option('display.max_rows', None)

sns.set_style("darkgrid")

from IPython.display import display, Markdown
from pprint import pprint 

DEBUG = True
SEED = 666

In [85]:
DATASET = "df_processed.pkl"
SCORE_DATASET = "df_score_processed.pkl"

import os, sys
COLAB = 'google.colab' in sys.modules
ROOT = "./"

if COLAB:
  from google.colab import drive
  if not os.path.isdir("/content/gdrive"):
    drive.mount("/content/gdrive")
    d = "/content/gdrive/MyDrive/datasets"
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
  if not os.path.isdir(ROOT): os.makedirs(ROOT)


def makedirs(d):
  if COLAB:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d)
  else:
    if not os.path.isdir(ROOT+d): os.makedirs(ROOT+d, mode=0o777, exist_ok=True)

for d in ['orig','data','output', 'submissions']: makedirs(d)

## Import Data & Features

In [86]:
df = pd.read_pickle(ROOT+"data/"+DATASET)
df_score = pd.read_pickle(ROOT+"data/"+SCORE_DATASET)

dfs = [df, df_score]

print(df.shape)
df.head()

(209672, 165)


,x000,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,x011,x012,x013,x014,x015,x016,x017,x018,x019,x020,x021,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x041,x042,x043,x044,x045,x046,x047,x048,x049,x050,x051,x052,x053,x054,x055,x056,x057,x058,x059,x060,x061,x062,x063,x064,x065,x066,x067,x068,x069,x070,x071,x072,x073,x074,x075,x076,x077,x078,x079,x080,x081,x082,x083,x084,x085,x086,x087,x088,x089,x090,x091,x092,x093,x094,x095,x096,x097,x098,x099,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,Target
0,0.577321,0,-1.528407,0.574591,-0.371609,0,0.769142,1,0,-0.564334,-1.670166,1,0.306430,0.430979,0.331989,0.560323,48,1.064115,0,2.123352,0.331989,3,0.822184,-1.670166,-1.636594,0.342279,-1.105716,0.925453,0.790756,-3.026960,33,-1.315302,0.696806,-0.371609,9,14,-1.105716,-0.986619,0.946460,0.925453,4,0.296706,0,0.553993,3,2.884178,5,0,2.123352,1.721216,-1.509301,-0.050747,-2.072471,1,0.812313,4.148009,0.948791,1.0,0.371670,1.064115,0.646152,0.907293,4,0,-1.038661,0.368816,-2.072471,2.422425,1,0.371670,0,234,0.0,19,0,5,-0.036157,0.815055,0.184478,3,0.548773,0.674836,0,0.315166,-0.842786,-2.302796,-0.099039,0.682440,3,1.324490,2.017993,1.324490,1,0.553993,-0.254312,-4.850562,0.077470,4,-0.242058,0.616705,0.577321,1,-3.975136,0.187446,0.368816,0.961387,9,6,2.024803,1,62,0,0.790756,0.436108,-2.839741,1.752665,0.882500,0.749183,-0.289017,0.187446,2.024803,0.0,0.738336,0,42,2,1,0.696806,1.721216,23,0.577970,-2.237272,39,1,0.0,2.648660,-1.322781,1,1,0.856375,0.674836,0.646152,-3.975136,-1.685895,0.856375,2.884178,-0.842786,-0.050747,-0.099039,-4.850562,1,-1.038661,69,0.749183,0.342279,-1.509301,2.422425,3,33,0.702792,0.548773,0.701321,-0.444765,10,Class_2
1,0.413242,0,-0.072692,0.567419,0.206837,0,-0.164559,0,0,-0.956555,4.030491,1,0.279233,0.512899,0.771122,0.183065,48,1.282373,0,-2.110657,0.771122,3,0.281779,4.030491,4.146307,0.393220,4.386999,0.528352,0.255444,2.837810,33,-7.434869,0.703253,0.206837,12,10,4.386999,-4.222262,0.347965,0.528352,5,-0.985570,0,0.354723,3,2.755122,27,8,-2.110657,4.238453,-0.097172,7.098692,0.803323,1,0.475270,-5.056917,0.328198,1.0,0.423728,1.282373,0.255066,0.493516,9,0,-5.380026,0.636534,0.803323,2.666922,1,0.423728,0,14,0.0,2,0,5,3.308104,0.359323,0.555923,2,1.914999,0.275613,0,0.639203,-0.820514,-3.699545,1.796063,0.795767,3,0.325121,-1.564933,0.325121,1,0.354723,-0.752444,-3.079449,1.073057,2,-1.637676,0.547283,0.413242,0,0.041940,0.495975,0.636534,0.283092,10,12,-1.353885,0,57,0,0.255444,0.367446,-0.256603,-0.603172,0.317201,0.908436,0.347056,0.495975,-1.353885,0.0,0.250319,10,141,2,0,0.703253,4.238453,45,0.365215,2.986686,25,1,0.0,-3.908599,-0.592003,1,1,-0.575467,0.275613,0.255066,0.041940,-3.751033,-0.575467,2.755122,-0.820514,7.098692,1.796063,-3.079449,1,-5.380026,54,0.908436,0.393220,-0.097172,2.666922,42,33,0.661232,1.914999,0.934367,0.524791,5,Class_2
2,-0.992723,0,5.089105,0.344148,1.052436,0,-1.066716,0,0,-0.852081,-5.396598,0,0.349402,0.541646,1.252255,1.696027,48,2.988087,3,-3.721994,1.252255,1,0.298688,-5.396598,0.240343,0.701801,-3.333315,0.331816,0.372463,1.002680,33,-7.104154,0.607222,1.052436,10,14,-3.333315,-4.765734,0.369349,0.331816,7,-2.538480,0,0.680543,3,-0.756906,25,0,-3.721994,1.923408,-1.279555,-1.710899,0.534373,0,0.455945,-2.741101,0.462819,0.0,0.419382,2.988087,0.618782,0.471865,9,5,5.444011,0.304979,0.534373,-2.901062,1,0.419382,0,150,0.0,2,0,5,0.002195,0.736656,0.572382,1,0.277789,0.379108,0,0.468445,-0.690706,-0.935914,-0.295098,0.683867,3,2.455901,1.120227,2.455901,1,0.680543,1.870748,0.884827,0.514067,9,-1.663241,0.478182,-0.992723,0,1.423420,0.821683,0.304979,0.762150,2,6,-0.507181,1,57,0,0.372463,0.332684,-2.889505,3.301401,0.630255,0.573692,-0.91956

In [87]:
with open(ROOT+"data/features.yaml") as file:
    yml_obj = yaml.load(file, Loader=yaml.FullLoader)

target = yml_obj["target"]
features = yml_obj["features"]
numerical_features = yml_obj["numerical_features"]
categorical_features = yml_obj["categorical_features"]

target_labels = sorted(df[target].unique())

print(f"Target: {target}")
print(f"Features: {features}")
print(f"Numerical Features: {numerical_features}")
print(f"Categorical Features: {categorical_features}")
print(f"Target Labels: {target_labels}")
print(f"Passed the sanity check: {len(numerical_features) + len(categorical_features) == len(features)}")

Target: Target
Features: ['x000', 'x001', 'x002', 'x003', 'x004', 'x005', 'x006', 'x007', 'x008', 'x009', 'x010', 'x011', 'x012', 'x013', 'x014', 'x015', 'x016', 'x017', 'x018', 'x019', 'x020', 'x021', 'x022', 'x023', 'x024', 'x025', 'x026', 'x027', 'x028', 'x029', 'x030', 'x031', 'x032', 'x033', 'x034', 'x035', 'x036', 'x037', 'x038', 'x039', 'x040', 'x041', 'x042', 'x043', 'x044', 'x045', 'x046', 'x047', 'x048', 'x049', 'x050', 'x051', 'x052', 'x053', 'x054', 'x055', 'x056', 'x057', 'x058', 'x059', 'x060', 'x061', 'x062', 'x063', 'x064', 'x065', 'x066', 'x067', 'x068', 'x069', 'x070', 'x071', 'x072', 'x073', 'x074', 'x075', 'x076', 'x077', 'x078', 'x079', 'x080', 'x081', 'x082', 'x083', 'x084', 'x085', 'x086', 'x087', 'x088', 'x089', 'x090', 'x091', 'x092', 'x093', 'x094', 'x095', 'x096', 'x097', 'x098', 'x099', 'x100', 'x101', 'x102', 'x103', 'x104', 'x105', 'x106', 'x107', 'x108', 'x109', 'x110', 'x111', 'x112', 'x113', 'x114', 'x115', 'x116', 'x117', 'x118', 'x119', 'x120', 'x121'

## Preperation

### Imports

In [88]:
# Imports

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

### Methods

In [89]:
def submission(y_pred_prob, name=f"submission-{pd.to_datetime('now', utc=True).strftime('%Y%m%d%H%M%S')}"):
    df_sub = pd.DataFrame(y_pred_prob, columns=target_labels)
    df_sub.index.name = 'id'
    df_sub.to_csv(ROOT+f'output/{name}.csv', index=True)
    print(f"Saved ({df_sub.shape[0]} rows) to: {ROOT}output/{name}.csv")

### Data Splitting

Data is being split into training and testing sets. The training set will be used to train the model, while the testing set will be used to evaluate the model.

The following dataframes are created:
- `X_train`: Features of the training set
- `X_test`: Features of the testing set
- `y_train`: Target of the training set
- `y_test`: Target of the testing set

Stratify is used to ensure that the target distribution is the same in both the training and testing sets. (We saw a small imbalance in the target distribution in the EDA notebook)

In [90]:
df_train , df_test = train_test_split(df, test_size=1-train_dataset_size, random_state=SEED, stratify=df[target])

df_train.sort_index(inplace=True)
df_test.sort_index(inplace=True)

print(df_train.shape, df_test.shape, df_score.shape)

(146770, 165) (62902, 165) (90000, 164)


### Scaling

The features are scaled using the `StandardScaler` from `sklearn.preprocessing`

In [91]:
ss = StandardScaler()
ss.fit(df_train[numerical_features])

x_train_num = ss.transform(df_train[numerical_features])
x_test_num = ss.transform(df_test[numerical_features])
x_score_num = ss.transform(df_score[numerical_features])

### Encoding

- The target feature is encoded using the `LabelEncoder` from `sklearn.preprocessing`
- The categorical features are encoded using the `OneHotEncoder` from `sklearn.preprocessing`

In [92]:
le = LabelEncoder()
le.fit(df[target])

y_train = le.transform(df_train[target])
y_test = le.transform(df_test[target])

In [93]:
ohe = OneHotEncoder()
ohe.fit(df_train[categorical_features])

x_train_cat = ohe.transform(df_train[categorical_features])
x_test_cat = ohe.transform(df_test[categorical_features])
x_score_cat = ohe.transform(df_score[categorical_features])

### Merge Encoded and Scaled Features

In [94]:
ohe_feature_names = ohe.get_feature_names_out(categorical_features)
column_names = numerical_features + list(ohe_feature_names)

x_train = pd.DataFrame(np.concatenate([x_train_num, x_train_cat.toarray()], axis=1), columns=column_names)
x_test = pd.DataFrame(np.concatenate([x_test_num, x_test_cat.toarray()], axis=1), columns=column_names)
x_score = pd.DataFrame(np.concatenate([x_score_num, x_score_cat.toarray()], axis=1), columns=column_names)

In [95]:
print(x_train.shape, x_test.shape, x_score.shape)

x_train.head()

(146770, 210) (62902, 210) (90000, 210)


,x000,x001,x002,x003,x004,x006,x009,x010,x012,x013,x014,x015,x016,x017,x019,x020,x022,x023,x024,x025,x026,x027,x028,x029,x030,x031,x032,x033,x034,x035,x036,x037,x038,x039,x040,x041,x042,x043,x044,x045,x046,x047,x048,x049,x050,x051,x052,x054,x055,x056,x058,x059,x060,x061,x062,x063,x064,x065,x066,x067,x069,x070,x071,x073,x075,x076,x077,x078,x079,x080,x081,x083,x084,x085,x086,x087,x089,x090,x091,x092,x093,x094,x095,x096,x097,x098,x099,x100,x102,x103,x104,x105,x106,x107,x108,x110,x112,x113,x114,x115,x116,x117,x118,x119,x120,x122,x123,x124,x127,x128,x129,x130,x131,x132,x135,x136,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,x005_0,x005_1,x005_2,x005_3,x007_0,x007_1,x008_0,x008_1,x011_0,x011_1,x018_0,x018_1,x018_2,x018_3,x021_0,x021_1,x021_2,x021_3,x053_0,x053_1,x057_0.0,x057_1.0,x068_0,x068_1,x068_2,x068_3,x072_0.0,x072_1.0,x072_2.0,x072_4.0,x074_0,x074_1,x082_0,x082_1,x082_2,x082_3,x088_0,x088_1,x088_2,x088_3,x101_0,x101_1,x109_0,x109_1,x111_0,x111_1,x111_2,x111_3,x121_0.0,x121_1.0,x125_0,x125_1,x125_2,x125_3,x126_0,x126_1,x133_0,x133_1,x133_2,x133_3,x134_0.0,x134_1.0,x137_0,x137_1,x138_0,x138_1,x150_0,x150_1,x150_2,x150_3
0,0.042872,-0.583120,-0.400982,0.536947,-0.372261,0.426217,-0.559965,-0.103054,-0.979858,-0.321422,0.331593,0.563381,0.446150,0.036221,0.937988,0.331102,1.374495,-0.103101,-0.572559,-0.722095,-0.488337,2.035537,1.419426,-1.607213,0.259882,-0.134652,0.843148,-0.371682,0.366085,0.643533,-0.488337,-0.056624,2.008644,2.038969,-0.531546,0.144128,-0.520944,0.212668,-0.305255,0.717429,-1.034122,-0.580772,0.939950,0.557853,-0.647653,0.198733,-2.062576,1.444731,1.253923,2.542888,-0.708314,0.036221,0.793591,1.946372,-2.546074,-0.634663,-0.064664,-0.665854,-2.062576,1.067703,-0.708314,-0.635177,0.780903,1.834379,0.130982,0.201459,1.389080,-1.434027,2.039174,0.548299,0.823022,-0.917457,-0.842968,-0.282526,-0.046349,0.752220,0.613345,1.027975,0.613345,-0.579478,0.212668,-0.256407,-1.226382,0.077450,-0.599310,0.146891,0.536241,0.042683,-1.763617,-1.346999,-0.665854,2.085832,-0.072773,-0.981914,1.004838,0.604023,1.419426,-0.273532,-1.254501,0.774298,1.429934,1.136642,0.109740,-1.346999,1.004838,1.025109,-2.643433,-0.902023,0.842518,0.557853,0.072587,0.598322,-0.960255,0.898148,1.175029,-1.324281,0.278000,0.821001,0.791583,-1.763617,-0.226341,0.278000,0.717429,-0.842968,0.198733,-0.046349,-1.226382,-0.064664,-0.986646,1.136642,-0.722095,-0.646503,1.067703,-1.931625,0.261596,0.959378,0.548195,0.796744,-0.450080,-0.012428,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,-0.065169,-0.583120,-1.101321,-1.105959,-0.435965,0.727085,-0.446196,-1.221691,-0.370583,1.366207,0.188622,-0.133686,0.446150,-0.062519,-0.149203,0.188281,-0.063061,-1.219623,-0.076806,2.019204,0.249086,-0.846156,-0.577369,-0.516290,0.259882,-0.627974,0.403718,-0.435241,0.897157,0.643533,0.249086,-0.547203,-1.302648,-0.847326,-1.189861,-2.599560,-0.520944,-1.576156,-0.305255,0.804848,-1.034122,2.685306,-0.149654,1.966958,0.264832,-1.250840,-0.987444,-1.575091,-0.425117,-1.261059,-0.781587,-0.062519,0.740141,0.765875,-0.211246,-0.150928,0.630232,-0.017150,-0.987444,-0.348051,-0.781587,-0.635177,0.220645,1.022919,-0.528986,-0.813536,-1.522772,2.120343,0.332128,0.145982,-0.359218,-0.671444,-0.383819,-0.341453,-0.223384,-1.238674,-0.418260,0.134810,-0.418260,-0.579478,-1.576156,-1.537012,-1.364101,-0.636506,-0.599310,0.758430,-0.101351,-0.065191,-0.224309,-1.012074,-0.017150,-0.985331,0.323151,0.714794,0.303956,0.604023,-0.577369,-0.434107,-0.036900,0.941395,-1.403035,-1.106446,0.926247,-1.012074,0.303956,-0.892922,-0.040085,1.072851,0.403438,1.966958,1.640211,-0.775040,-0.203463,1.211307,0.138178,0.996478,-0.455436,-0.358327,0.738265,-0.224309,0.202010,-0.455436,0.8

## Build Models

### Imports

May require XGBoost to be installed. Uncomment the following line to install it.
I used the following command to install XGBoost:
```bash
conda install xgboost
```

In [96]:
# classification models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# pickle
import pickle

# metrics
from sklearn.metrics import log_loss, confusion_matrix, classification_report


In [97]:
classifiers = {}

In [98]:
def setup_model(name, classifier, x_train, y_train):
    if should_build_model:
        print(f"Training {name}...")
        start_time = pd.Timestamp.now()
        classifier.fit(x_train, y_train)
        end_time = pd.Timestamp.now()
        print(f"Training {name} took: {end_time - start_time}")

        if should_save_model:
            with open(ROOT+f"output/{name}.pkl", 'wb') as file:
                pickle.dump(classifier, file)
                print(f"Saved model to: {ROOT}output/{name}.pkl")
                
    else:
        with open(ROOT+f"output/{name}.pkl", 'rb') as file:
            classifier = pickle.load(file)
            print(f"Loaded model from: `{ROOT}output/{name}.pkl`")

    return classifier

In [99]:
def evaluate_model(name, classifier, x_test, y_test):
    # evaluate model
    # y_pred = classifier.predict(x_test)
    y_pred_prob = classifier.predict_proba(x_test)

    # print(f"Confusion Matrix for {name}:")
    # print(confusion_matrix(y_test, y_pred))
    # print(f"Classification Report for {name}:")
    # print(classification_report(y_test, y_pred, target_names=target_labels))
    print(f"Log Loss for {name}: {log_loss(y_test, y_pred_prob)}")

In [100]:
classifier_name = "Logistic_Regression"
classifiers.update({
    classifier_name : LogisticRegression(random_state=SEED, max_iter=1000, n_jobs=-1),
})

classifiers[classifier_name] = setup_model(classifier_name, classifiers[classifier_name], x_train, y_train)

Loaded model from: `./output/Logistic_Regression.pkl`


In [101]:
classifier_name = "Random_Forest_(max_depth=10)"
classifiers.update({
    classifier_name : RandomForestClassifier(random_state=SEED, n_jobs=-1, max_depth=10),
})

classifiers[classifier_name] = setup_model(classifier_name, classifiers[classifier_name], x_train, y_train)

Loaded model from: `./output/Random_Forest_(max_depth=10).pkl`


In [102]:
## TOOK TOO LONG TO TRAIN 14+ minutes

# classifier_name = "Gradient Boosting"
# classifiers.update({
#     classifier_name : GradientBoostingClassifier(random_state=SEED),
# })

# classifiers[classifier_name] = setup_model(classifier_name, classifiers[classifier_name], x_train, y_train)

In [103]:
## TOOK TOO LONG TO TRAIN 18+ minutes

# classifier_name = "Support Vector Machine"
# classifiers.update({
#     classifier_name : SVC(probability=True, random_state=SEED),
# })

# classifiers[classifier_name] = setup_model(classifier_name, classifiers[classifier_name], x_train, y_train)

In [104]:
classifier_name = "Neural_Network"
classifiers.update({
    classifier_name : MLPClassifier(random_state=SEED),
})

classifiers[classifier_name] = setup_model(classifier_name, classifiers[classifier_name], x_train, y_train)

Loaded model from: `./output/Neural_Network.pkl`


In [105]:
classifier_name = "Naive_Bayes"
classifiers.update({
    classifier_name : GaussianNB(),
})

classifiers[classifier_name] = setup_model(classifier_name, classifiers[classifier_name], x_train, y_train)

Loaded model from: `./output/Naive_Bayes.pkl`


In [106]:
classifier_name = "K-Nearest_Neighbors_(3)"
classifiers.update({
    classifier_name : KNeighborsClassifier(n_jobs=-1, n_neighbors=3),
})

classifiers[classifier_name] = setup_model(classifier_name, classifiers[classifier_name], x_train, y_train)

Loaded model from: `./output/K-Nearest_Neighbors_(3).pkl`


In [107]:
classifier_name = "K-Nearest_Neighbors_(5)"
classifiers.update({
    classifier_name : KNeighborsClassifier(n_jobs=-1, n_neighbors=5),
})

classifiers[classifier_name] = setup_model(classifier_name, classifiers[classifier_name], x_train, y_train)

Loaded model from: `./output/K-Nearest_Neighbors_(5).pkl`


In [108]:
classifier_name = "K-Nearest_Neighbors_(7)"
classifiers.update({
    classifier_name : KNeighborsClassifier(n_jobs=-1, n_neighbors=7),
})

classifiers[classifier_name] = setup_model(classifier_name, classifiers[classifier_name], x_train, y_train)

Loaded model from: `./output/K-Nearest_Neighbors_(7).pkl`


In [109]:
classifier_name = "XGBoost"
classifiers.update({
    classifier_name : XGBClassifier(random_state=SEED),
})

classifiers[classifier_name] = setup_model(classifier_name, classifiers[classifier_name], x_train, y_train)

Loaded model from: `./output/XGBoost.pkl`


## Evaluate Models

In [110]:
def evaluate_model(name, classifier, x_test, y_test):
    y_pred_prob = classifier.predict_proba(x_test)
    log_loss_score = log_loss(y_test, y_pred_prob)
    print(f"Log Loss for {name}: {log_loss_score}")
    return log_loss_score

In [111]:
(best_performing_model, best_performing_model_score) = (None, np.inf)

for classifier_name, classifier in classifiers.items():
    score = evaluate_model(classifier_name, classifier, x_test, y_test)
    if score < best_performing_model_score:
        (best_performing_model, best_performing_model_score) = (classifier_name, score)

Log Loss for Logistic_Regression: 1.4304415042849623
Log Loss for Random_Forest_(max_depth=10): 1.3827452278686074
Log Loss for Neural_Network: 1.2325728655506123


Log Loss for Naive_Bayes: 3.3207248885613687
Log Loss for K-Nearest_Neighbors_(3): 15.938307833213829
Log Loss for K-Nearest_Neighbors_(5): 10.126555002534053
Log Loss for K-Nearest_Neighbors_(7): 6.848397781180832
Log Loss for XGBoost: 1.2005250587967873


In [112]:
print(f"Best performing model: {best_performing_model} with a log loss of {best_performing_model_score}")

Best performing model: XGBoost with a log loss of 1.2005250587967873


## Generate Submission File

In [113]:
target_labels = sorted(df[target].unique())
target_labels

['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5']

In [114]:
def submission(y_pred_prob, name=f"submission-{pd.to_datetime('now', utc=True).strftime('%Y%m%d%H%M%S')}"):
    df_sub = pd.DataFrame(y_pred_prob, columns=target_labels)
    df_sub.index.name = 'id'
    df_sub.to_csv(ROOT+f'submissions/{name}.csv', index=True)
    print(f"Saved ({df_sub.shape[0]} rows) to: {ROOT}submissions/{name}.csv")

In [115]:
best_performing_model = classifiers[best_performing_model]
y_score = best_performing_model.predict_proba(x_score)
submission(y_score)

Saved (90000 rows) to: ./submissions/submission-20240323213943.csv


# Notes